# Part I. ETL Pipeline for Pre-Processing the Files

PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import os
import csv
import glob
from cassandra.cluster import Cluster

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print('The current working directory:', os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/data/event_data'

# Create a for loop to create a list of files and collect each filepath
file_path_list = ''
for root, dirs, files in os.walk(filepath):
    
    # Join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    
    # Sort the collected list of paths
    file_path_list.sort()
    
    # Print the total number of paths collected
    print('Total filepaths collected:', len(file_path_list))
    
    # Print the list of filepaths that have been collected
    for count, path in enumerate(file_path_list):
        print(count+1, path)
    
    # Ignore the .ipynb_checkpoints hidden subdirectory
    break

The current working directory: /home/workspace
Total filepaths collected: 30
1 /home/workspace/data/event_data/2018-11-01-events.csv
2 /home/workspace/data/event_data/2018-11-02-events.csv
3 /home/workspace/data/event_data/2018-11-03-events.csv
4 /home/workspace/data/event_data/2018-11-04-events.csv
5 /home/workspace/data/event_data/2018-11-05-events.csv
6 /home/workspace/data/event_data/2018-11-06-events.csv
7 /home/workspace/data/event_data/2018-11-07-events.csv
8 /home/workspace/data/event_data/2018-11-08-events.csv
9 /home/workspace/data/event_data/2018-11-09-events.csv
10 /home/workspace/data/event_data/2018-11-10-events.csv
11 /home/workspace/data/event_data/2018-11-11-events.csv
12 /home/workspace/data/event_data/2018-11-12-events.csv
13 /home/workspace/data/event_data/2018-11-13-events.csv
14 /home/workspace/data/event_data/2018-11-14-events.csv
15 /home/workspace/data/event_data/2018-11-15-events.csv
16 /home/workspace/data/event_data/2018-11-16-events.csv
17 /home/workspace/d

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the filepath list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# Get the total number of rows 
print('The total number of rows: ', len(full_data_rows_list))

# Creating a smaller event data csv file called event_datafile_new.csv that will 
# be used to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName',\
                     'length','level','location','sessionId','song','userId'])
    
    # Only include data that contains an artist name
    count = 0
    for row in full_data_rows_list:
        if (row[0] == ''):
            count += 1
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))
    
    print('The total number of excluded rows:', count)

The total number of rows:  8056
The total number of excluded rows: 1236


In [4]:
# Check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="assets/images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb
    WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
                    """)
except Exception as e:
    print (e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print (e)


Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

### Create queries to ask the following three questions of the data

1. **Query 1:** Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


2. **Query 2:** Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182


3. **Query 3:** Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
# Query 1
# Create Table
query = "CREATE TABLE IF NOT EXISTS session_item_events "
query += "(sessionId int, itemInSession int, \
             artist varchar, song varchar, length float, \
             PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print (e)

In [9]:
# Insert data from CSV
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    
    # Skip header
    columns = next(csvreader)
        
    for line in csvreader:
        
        # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO session_item_events "
        query += "(sessionId, itemInSession, artist, song, length)"
        query += "VALUES (%s, %s, %s, %s, %s)"

        # Assign which column element should be assigned for each column in the INSERT statement.
        try:
            session.execute(query, (int(line[columns.index("sessionId")]), 
                                int(line[columns.index("itemInSession")]),
                                line[columns.index("artist")], 
                                line[columns.index("song")], 
                                float(line[columns.index("length")])))
        except Exception as e:
            print(e)

#### Do a SELECT to verify that the data have been inserted into each table

For table session_item_events, the sessionId was used as a partition key because the queries will filter by this column. The itemInSession was used as a clustering column to help make up a unique key.

In [10]:
# Add in the SELECT statement to verify the data was entered into the table
query = "SELECT artist, song, length from session_item_events "
query += "WHERE sessionId = 338 and itemInSession = 4 ;"

try:
    rows = session.execute(query)
except Exception as e:
    print (e)
    
print ('----------------')
for row in rows:
    print ("Artist:", row.artist)
    print ("Song:", row.song)
    print ("Length:", row.length)
    print ('----------------')

----------------
Artist: Faithless
Song: Music Matters (Mark Knight Dub)
Length: 495.30731201171875
----------------


### REPEAT THE ABOVE THREE CELLS FOR THE REMAINING QUERYS

In [11]:
# Query 2
# Create Table
query = "CREATE TABLE IF NOT EXISTS user_session_item_events "
query += "(userId int, sessionId int, itemInSession int, \
             firstName varchar, lastName varchar, \
             artist varchar, song varchar, PRIMARY KEY \
             (userId, sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print (e)

In [12]:
# Insert data from previous CSV
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    
    # Skip header
    columns = next(csvreader)
        
    for line in csvreader:
        
        # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_session_item_events "
        query += "(userId, sessionId, itemInSession, firstName, lastName, artist, song)"
        query += "VALUES (%s, %s, %s, %s, %s, %s, %s)"

        # Assign which column element should be assigned for each column in the INSERT statement.
        try:
            session.execute(query, (int(line[columns.index("userId")]), 
                                    int(line[columns.index("sessionId")]), 
                                    int(line[columns.index("itemInSession")]), 
                                    line[columns.index("firstName")], 
                                    line[columns.index("lastName")], 
                                    line[columns.index("artist")], 
                                    line[columns.index("song")]))
        except Exception as e:
            print(e)

For table user_session_item_events, the userId was used as a partition key because the queries will filter by this column. The sessionId and itemInSession were used as clustering columns to help make up a unique key.

In [13]:
# Add in the SELECT statement to verify the data was entered into the table
query = "SELECT artist, song, firstName, lastName from user_session_item_events "
query += "WHERE userId = 10 and sessionId = 182 ;"

try:
    rows = session.execute(query)
except Exception as e:
    print (e)

print ('----------------')
for row in rows:
    print ("Artist:", row.artist)
    print ("Song", row.song)
    print ("First Name:", row.firstname)
    print ("Last Name:", row.lastname)
    print ('----------------')

----------------
Artist: Down To The Bone
Song Keep On Keepin' On
First Name: Sylvie
Last Name: Cruz
----------------
Artist: Three Drives
Song Greece 2000
First Name: Sylvie
Last Name: Cruz
----------------
Artist: Sebastien Tellier
Song Kilometer
First Name: Sylvie
Last Name: Cruz
----------------
Artist: Lonnie Gordon
Song Catch You Baby (Steve Pitron & Max Sanna Radio Edit)
First Name: Sylvie
Last Name: Cruz
----------------


In [14]:
# Query 3
# Create Table
query = "CREATE TABLE IF NOT EXISTS song_user_events "
query += "(song varchar, userId int, \
             firstName varchar, lastName varchar, \
             PRIMARY KEY (song, userId))"

try:
    session.execute(query)
except Exception as e:
    print (e)

In [15]:
# Insert data from previous CSV
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    
    # Skip header
    columns = next(csvreader)
        
    for line in csvreader:
        
        # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_user_events "
        query += "(song, userId, firstName, lastName) "
        query += "VALUES (%s, %s, %s, %s)"

        # Assign which column element should be assigned for each column in the INSERT statement.
        try:
            session.execute(query, (line[columns.index("song")], 
                                    int(line[columns.index("userId")]), 
                                    line[columns.index("firstName")], 
                                    line[columns.index("lastName")]))
        except Exception as e:
            print(e)

For table song_user_events, the song was used as a partition key because the queries will filter by this column. The userId length was used as a clustering column to help make up a unique key.

In [16]:
# Add in the SELECT statement to verify the data was entered into the table
query = "SELECT firstName, lastName from song_user_events "
query += "WHERE song = 'All Hands Against His Own' ;"

try:
    rows = session.execute(query)
except Exception as e:
    print (e)
    
print ('----------------')
for row in rows:
    print ("First Name:", row.firstname)
    print ("Last Name:", row.lastname)
    print ('----------------')

----------------
First Name: Jacqueline
Last Name: Lynch
----------------
First Name: Tegan
Last Name: Levine
----------------
First Name: Sara
Last Name: Johnson
----------------


### Drop the tables before closing out the sessions

In [17]:
# Drop the table before closing out the sessions

In [18]:
session.execute('DROP TABLE session_item_events');
session.execute('DROP TABLE user_session_item_events');
session.execute('DROP TABLE song_user_events');

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()